In [1]:
from dateutil.relativedelta import relativedelta
import mysql.connector
import datetime
import requests
from acruxdb import ParceirosDB

po = ParceirosDB()


HEADERS = {
        "accept": "application/json",
        "Authorization": "Bearer 6|EbVPDgxM5BhGdbsU6MzB9y7LePsfvXIdd3d6NMGC"
    }
HOST = '35.237.180.90'
#HOST = '10.7.32.12'

In [1]:
import pandas as pd

# Criando um DataFrame de exemplo
data = pd.DataFrame({
    "cpf": ["123456789", "987654321"],
    "nome": ["João", "Maria"],
    "telefone": ["99999-9999", "88888-8888"],
    "idade": [25, float('nan')]  # Simulando um valor NaN
})

# Substituir NaN por None antes de converter para dicionário
data = data.where(pd.notna(data), None)

# Converter para lista de dicionários
records = data.to_dict(orient="records")

print(records)


[{'cpf': '123456789', 'nome': 'João', 'telefone': '99999-9999', 'idade': 25.0}, {'cpf': '987654321', 'nome': 'Maria', 'telefone': '88888-8888', 'idade': nan}]


In [2]:
data.drop_duplicates()

,cpf,nome,telefone,idade
0,123456789,João,99999-9999,25.0
1,987654321,Maria,88888-8888,NaN


In [2]:
a = 'tesRS'
a.lower()

'tesrs'

In [2]:
po.consulta_campos()

['id',
 'cpf',
 'nome',
 'sexo',
 'estado_civil',
 'data_nascimento',
 'uf_nascimento',
 'nome_pai',
 'nome_mae',
 'ajuizado',
 'central',
 'acordo_limpanomes',
 'acordo_saiuacordo',
 'enderecos',
 'emails',
 'contratos',
 'telefones']

In [3]:
maiordesconto = 200
valor_acordo  = 120

diff = valor_acordo - maiordesconto
print(diff < -100)


False


In [23]:
def corrige_data(data):
    data = data.split('/')
    return f'{data[2]}-{data[1]}-{data[0]}'

class AcruxDB:
    
    def __init__(self):
        self.host     =  HOST
        self.database = 'prd-resolvecontas'
        self.user     = 'prd_user_powerbi'
        self.password = '("rYLs+nER0i&Trc'
        
    def conecta_db(self):
        conn = mysql.connector.connect(
                host=self.host, 
                database=self.database, 
                user=self.user, 
                password=self.password
            )
        return conn
    
    
    def retornaContratosPessoaUnico(self, cpf, contrato):
        
        try:
            conn = self.conecta_db()
            cursor = conn.cursor(dictionary=True)
            QUERY = f"""
                           SELECT id FROM contrato WHERE contrato = '{contrato}' AND  RIGHT(cpf_devedor, 11) = '{cpf}' LIMIT 1  
                    """
            cursor.execute(QUERY)
            contratos = cursor.fetchall()
        except Exception as e:
            print(e)
            return None
        conn.close()
        try:
            id = contratos[0]['id']
        except:
            id = None
        return id
    
    
    def corrige_parcelas(self, parcelas, id_acordo):
        
        try:
            for parcela in parcelas:
                QUERY = f"UPDATE resolvecontas_parcela SET dataVencimento = '{corrige_data(parcela['dataVencimento'])}' WHERE acordo_id = {id_acordo} AND numeroParcela = {parcela['numero']};"    
                conn = self.conecta_db()
                cursor = conn.cursor()
                cursor.execute(QUERY)
                conn.commit()
                conn.close()
        except Exception as e:
            print(e)
            return False
        return True

In [24]:
PARCELAS = '[{"valor": "170,18", "numero": "1", "dataVencimento": "22/05/2024"}, {"valor": "170,18", "numero": "2", "dataVencimento": "25/06/2024"}, {"valor": "170,18", "numero": "3", "dataVencimento": "25/07/2024"}, {"valor": "170,18", "numero": "4", "dataVencimento": "25/08/2024"}, {"valor": "170,18", "numero": "5", "dataVencimento": "25/09/2024"}, {"valor": "170,18", "numero": "6", "dataVencimento": "25/10/2024"}, {"valor": "170,18", "numero": "7", "dataVencimento": "25/11/2024"}, {"valor": "170,18", "numero": "8", "dataVencimento": "25/12/2024"}, {"valor": "170,18", "numero": "9", "dataVencimento": "25/01/2025"}, {"valor": "170,18", "numero": "10", "dataVencimento": "25/02/2025"}, {"valor": "170,18", "numero": "11", "dataVencimento": "25/03/2025"}]'

In [25]:
PARCELAS = eval(PARCELAS)

In [26]:
db = AcruxDB()
db.corrige_parcelas(PARCELAS, 19242)

True

In [12]:
QUERY

"UPDATE resolvecontas_parcela SET dataVencimento = '2024-05-22' WHERE acordo_id = 7976 AND numeroParcela = 1;UPDATE resolvecontas_parcela SET dataVencimento = '2024-06-25' WHERE acordo_id = 7976 AND numeroParcela = 2;UPDATE resolvecontas_parcela SET dataVencimento = '2024-07-25' WHERE acordo_id = 7976 AND numeroParcela = 3;UPDATE resolvecontas_parcela SET dataVencimento = '2024-08-25' WHERE acordo_id = 7976 AND numeroParcela = 4;UPDATE resolvecontas_parcela SET dataVencimento = '2024-09-25' WHERE acordo_id = 7976 AND numeroParcela = 5;UPDATE resolvecontas_parcela SET dataVencimento = '2024-10-25' WHERE acordo_id = 7976 AND numeroParcela = 6;UPDATE resolvecontas_parcela SET dataVencimento = '2024-11-25' WHERE acordo_id = 7976 AND numeroParcela = 7;UPDATE resolvecontas_parcela SET dataVencimento = '2024-12-25' WHERE acordo_id = 7976 AND numeroParcela = 8;UPDATE resolvecontas_parcela SET dataVencimento = '2025-01-25' WHERE acordo_id = 7976 AND numeroParcela = 9;UPDATE resolvecontas_parcel

In [1]:
from acruxdb import ParceirosDB

In [ ]:


po = ParceirosDB()
with open('teste.sql', 'r') as file:
    conte = file.read()
    campos_padrao = po.consulta_pessoas(conte)
    print(campos_padrao)


    id                              nome          cpf email telefone  \
0  110                          anderson  05726067703  None     None   
1  111                             maria  14699705707  None     None   
2  113  Lucas Oliveira Ferreira de Matos  14525757701  None     None   
3  114                     Luiz Fernando  17854401799  None     None   

  data_nascimento  uf endereco nome_pai nome_mae  sexo estado_civil  \
0      1978-09-05  SP     None     None     None  None         None   
1      1980-06-20  RS     None     None     None  None         None   
2      1996-03-10  SP     None     None     None  None         None   
3      1998-11-20  RJ     None     None     None  None         None   

  custom_fields  empresa_id  
0          null           9  
1          null           9  
2          null           9  
3          null           9  
